# Introduction to LangChain

In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

## Prerequisites

You will need to provision the following Azure resources:
* Azure OpenAI with two models: GPT-4o and Embedding model.
* Azure AI Search.
You can run the terraform template from folder `../400_azure_ai_foundry` to create all of these resources by simply running the following commands.

```sh
terraform init
terraform plan -out tfplan
terraform apply tfplan
```

Then you will need to setup the values for the environment variables in file `.env`.

The following resources should be created.

![](images/resources.png)

To install LangChain run:

In [1]:
%pip install -qU "langchain[openai]"
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Let's learn how to use a language model by itself. LangChain supports many different language models like OpenAI, Anthropic, Mistral, etc, that you can use interchangeably.

In [ ]:
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os
import json

if os.path.exists(".env"):
    load_dotenv(override=True)

model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

response = model.invoke("Hello, world!")

# format json response
json_formatted_str = json.dumps(response.to_json(), indent=4)
print(json_formatted_str)

{
    "lc": 1,
    "type": "constructor",
    "id": [
        "langchain",
        "schema",
        "messages",
        "AIMessage"
    ],
    "kwargs": {
        "content": "Hello, world! \ud83c\udf0d How can I assist you today?",
        "additional_kwargs": {
            "refusal": null
        },
        "response_metadata": {
            "token_usage": {
                "completion_tokens": 13,
                "prompt_tokens": 11,
                "total_tokens": 24,
                "completion_tokens_details": {
                    "accepted_prediction_tokens": 0,
                    "audio_tokens": 0,
                    "reasoning_tokens": 0,
                    "rejected_prediction_tokens": 0
                },
                "prompt_tokens_details": {
                    "audio_tokens": 0,
                    "cached_tokens": 0
                }
            },
            "model_name": "gpt-4o-2024-11-20",
            "system_fingerprint": "fp_b705f0c291",
            "promp

Use a message with system and human input.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

response = model.invoke(messages)

# format json response
json_formatted_str = json.dumps(response.to_json(), indent=4)
print(json_formatted_str)

{
    "lc": 1,
    "type": "constructor",
    "id": [
        "langchain",
        "schema",
        "messages",
        "AIMessage"
    ],
    "kwargs": {
        "content": "Ciao!",
        "additional_kwargs": {
            "refusal": null
        },
        "response_metadata": {
            "token_usage": {
                "completion_tokens": 3,
                "prompt_tokens": 20,
                "total_tokens": 23,
                "completion_tokens_details": {
                    "accepted_prediction_tokens": 0,
                    "audio_tokens": 0,
                    "reasoning_tokens": 0,
                    "rejected_prediction_tokens": 0
                },
                "prompt_tokens_details": {
                    "audio_tokens": 0,
                    "cached_tokens": 0
                }
            },
            "model_name": "gpt-4o-2024-11-20",
            "system_fingerprint": "fp_b705f0c291",
            "prompt_filter_results": [
                {
           

Print just the response from the model.

In [21]:
print(response.content)

Ciao!


LangChain also supports chat model inputs via strings or OpenAI format. The following are equivalent:
```python
model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Hello")])
```

Stream the response.

In [22]:
for token in model.stream(messages):
    print(token.content, end="|")

||C|iao|!||

## Using Prompt Templates in LangChain

### Defining the template.

In [23]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

Creating a prompt from a template.

In [24]:
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

View the messages.

In [25]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

Invoke the chat model on the formatted prompt.

In [26]:
response = model.invoke(prompt)
print(response.content)

Ciao!


### More resources

https://python.langchain.com/docs/tutorials/llm_chain/

https://python.langchain.com/docs/how_to/document_loader_pdf/

https://python.langchain.com/docs/how_to/semantic-chunker/

https://langchain-opentutorial.gitbook.io/langchain-opentutorial/07-textsplitter/04-semanticchunker